A companion to:
[Transit Data Quality Analysis Standard Operating Procedures ](https://caltrans.sharepoint.com/:w:/s/DOTPMPHQ-DataandDigitalServices/Ec2SNnePpLlHsSayRT-ptc0B0JbEtVxRKrz-3lWG26v_JQ?e=cx8QUr)


In [3]:
from dotenv import load_dotenv
import json
import os

import pandas as pd
import requests

from calitp_data_analysis.sql import to_snakecase
import script

In [2]:
load_dotenv()
MTC_API_KEY = os.getenv('MTC_API_KEY')
mtc_operator_url = f"https://api.511.org/transit/operators?api_key={MTC_API_KEY}"

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

## Airtable

In [3]:
airtable = script.load_airtable(
    "./Transit Data Quality Issues-Open Issues by Distrcit.csv"
)

## Catastrophic

In [4]:
cat_df = script.load_catastrophic_errors(
    "./feed_info_expired_feeds_2024-01-19T16_26_39.694494Z.xlsx"
)

/opt/conda/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [5]:
cat_df

,service_name,district,uri,date
1,GTrans,7,http://ridegtrans.com/gtfs.zip,2024-01-18
5,GTrans,7,http://ridegtrans.com/gtfs.zip,2024-01-17
8,Commerce Municipal Bus Lines,7,https://citycommbus.com/gtfs,2024-01-16
13,Commerce Municipal Bus Lines,7,https://citycommbus.com/gtfs,2024-01-15
18,Commerce Municipal Bus Lines,7,https://citycommbus.com/gtfs,2024-01-14
23,Commerce Municipal Bus Lines,7,https://citycommbus.com/gtfs,2024-01-13
27,Commerce Municipal Bus Lines,7,https://citycommbus.com/gtfs,2024-01-12
31,Commerce Municipal Bus Lines,7,https://citycommbus.com/gtfs,2024-01-11
12,Burbank Bus,7,https://rideburbankbus.com/gtfs,2024-01-15
17,Burbank Bus,7,https://rideburbankbus.com/gtfs,2024-01-14


In [6]:
script.summarize_cat(cat_df, airtable)

,service_name,uri,# of days with expired feed
0,Burbank Bus,https://rideburbankbus.com/gtfs,3
1,Commerce Municipal Bus Lines,https://citycommbus.com/gtfs,6
2,GTrans,http://ridegtrans.com/gtfs.zip,2


,service_name,uri,# of days with expired feed,gtfs_datasets,services,issue_type,description,airtable_ticket
0,Burbank Bus,https://rideburbankbus.com/gtfs,3,NaN,NaN,NaN,NaN,NaN
1,Commerce Municipal Bus Lines,https://citycommbus.com/gtfs,6,NaN,NaN,NaN,NaN,NaN
2,GTrans,http://ridegtrans.com/gtfs.zip,2,None,GTrans,New GTFS Realtime System Setup,GTrans was contacted as a part of issue #76 and said they would like to chat about something related to GTFS-RT data that they are in the process of setting up.,Yes


## Incompleteness

In [4]:
r = requests.get(mtc_operator_url)
df_511 = pd.DataFrame(json.loads(r.content))

Bay Area feeds that Real Time feeds are monitored

In [7]:
df_511 = pd.DataFrame(json.loads(r.content))
df_511.loc[df_511['Monitored'].values,][['Name']]

,Name
4,AC TRANSIT
7,Bay Area Rapid Transit
8,Caltrain
12,County Connection
13,Dumbarton Express Consortium
15,FAST
17,Golden Gate Transit
18,Livermore Amador Valley Transit Authority
19,Marin Transit
21,MVgo


In [8]:
airtable.loc[airtable.issue_type.str.contains("Completeness")]

,gtfs_datasets,services,issue_type,description,airtable_ticket
7,"Bay Area 511 Emery Go-Round VehiclePositions,Bay Area 511 Emery Go-Round TripUpdates",Emery Go-Round,GTFS Realtime Completeness Problem,"As of December 15, Bay Area 511 Emery Go-Round GTFS-RT has recorded 0% in the past 14 days. Since November 30, TripUpdates and VehiclePositions has reported 0% trip data.",Yes
8,Bay Area 511 Petaluma VehiclePositions,Petaluma Transit,GTFS Realtime Completeness Problem,"Vehicle Position real-time data production has been consistently inconsistent throughout the last month. The average of the last two weeks Vehicle Position data production is 34.63%, which is below the threshold.",Yes
9,Bay Area 511 Union City Transit VehiclePositions,Union City Transit,GTFS Realtime Completeness Problem,"In the past 14 days, GTFS-RT Vehicle Positions is 37.21%, which is a reduction from 46.15% VP recorded on August 25, 2023. Also, September GTFS-RT Vehicle Positions activity has fluctuated from 31.41% to 41.67%.\n\n",Yes
10,Bay Area 511 Vine Transit VehiclePositions,Vine Transit,GTFS Realtime Completeness Problem,GTFS-Realtime Vehicle Position data production has not been consistent throughout the month. The last two weeks' average Vehicle Position data production was 39.86%,Yes
11,Bear Schedule,Bear Transit,GTFS Realtime Completeness Problem,"As of January 5, Bear Schedule recorded 20.54% TripUpdates and 18.95% VehiclePositions in the past 14 days. From December 23 through January 2, 0% of trip data for TripUpdates and VehiclePosition was reported. However on January 3, both TripUpdates and VehiclePosition reported over 70% of trip data.",Yes
19,Desert Roadrunner GMV Schedule,Desert Roadrunner,GTFS Realtime Completeness Problem,"As of January 5, Desert Roadrunner GMV Schedule recorded 21.78% TripUpdates and 20.83% VehiclePositions in the past 14 days. From December 25 through January 4, TripUpdates and VehiclePosition fluctuated and reported less than 44% of trip data.",Yes
35,"MVGO VehiclePositions,MVGO TripUpdates",MVGO,GTFS Realtime Completeness Problem,Since July 3 there have been zero trips accounted for in both the Vehicle Positions and Trip Updates for MVGO,Yes
37,Merced VehiclePositions,Merced The Bus,GTFS Realtime Completeness Problem,Transit Joint Powers Authority for Merced County: GTFS-RT issue: complete outage of GTFS-RT Vehicle Positions Data,Yes
38,"Montebello Vehicle Positions,Montebello TripUpdates",Montebello Bus Lines,GTFS Realtime Completeness Problem,"As of December 15, Montebello GTFS-RT has recorded 23.47% TripUpdates and 18.42% VehiclePositions in the past 14 days. Since December 11, TripUpdates and VehiclePosition has reported 0% trip data.",Yes
43,"Santa Ynez Vehicle Positions,Santa Ynez Mecatran Schedule",Santa Ynez Valley Transit,GTFS Realtime Completeness Problem,"As of 11/16: Only 39% of trips have vehicle position messages, but TU is ok.",Yes


In [9]:
script.incomplete(
    "./gtfs_rt_trip_updates_completeness__last_14_days__2024-01-19T16_26_21.805336Z.xlsx",
    "./gtfs_rt_vehicle_positions_completeness__last_14_days__2024-01-19T16_26_25.833257Z.xlsx",
    airtable,
    41,
)

/opt/conda/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,name,%_of_trips_with_tu_messages,%_of_trips_with_vp_messages,gtfs_datasets,services,issue_type,description,airtable_ticket,_merge
0,Anaheim Resort Schedule,14.76,12.50,Anaheim Resort Schedule,Anaheim Resort Transportation,Service Accuracy,Anaheim Transportation Network: GTFS-Schedule issue: thousands of duplicate trips on Toy Story Line,Yes,both
1,Bay Area 511 Angel Island-Tiburon Ferry Schedule,0.00,0.00,NA,NA,NA,NA,NA,found_in_tu_only
2,Bay Area 511 BART Schedule,OK,0.00,NA,NA,NA,NA,NA,found_in_tu_only
3,Bay Area 511 Capitol Corridor Schedule,40.42,40.42,NA,NA,NA,NA,NA,found_in_tu_only
4,Bay Area 511 Commute.org Schedule,0.00,0.00,NA,NA,NA,NA,NA,found_in_tu_only
5,Bay Area 511 Emery Go-Round Schedule,0.00,0.00,NA,NA,NA,NA,NA,found_in_tu_only
6,Bay Area 511 Golden Gate Ferry Schedule,0.00,0.00,NA,NA,NA,NA,NA,found_in_tu_only
7,Bay Area 511 MVGO Schedule,25.00,25.00,NA,NA,NA,NA,NA,found_in_tu_only
8,Bay Area 511 Mission Bay Schedule,0.00,0.00,NA,NA,NA,NA,NA,found_in_tu_only
9,Bay Area 511 Rio Vista Delta Breeze Schedule,0.00,0.00,NA,NA,NA,NA,NA,found_in_tu_only


In [10]:
ops_to_check = [
    "Santa Ynez Mecatran Schedule",
    "SunLine Avail Schedule",
    "Bear Schedule",
    "Montebello Schedule",
]

In [11]:
# Unfiltered version of the incompleteness dataq
incompleteness_complete = script.incomplete(
    "./gtfs_rt_trip_updates_completeness__last_14_days__2024-01-19T16_26_21.805336Z.xlsx",
    "./gtfs_rt_vehicle_positions_completeness__last_14_days__2024-01-19T16_26_25.833257Z.xlsx",
    airtable,
    100,
)

/opt/conda/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [12]:
incompleteness_complete.loc[incompleteness_complete.name.isin(ops_to_check)]

,name,%_of_trips_with_tu_messages,%_of_trips_with_vp_messages,gtfs_datasets,services,issue_type,description,airtable_ticket,_merge
45,Bear Schedule,74.44,69.89,Bear Schedule,Bear Transit,GTFS Realtime Completeness Problem,"As of January 5, Bear Schedule recorded 20.54% TripUpdates and 18.95% VehiclePositions in the past 14 days. From December 23 through January 2, 0% of trip data for TripUpdates and VehiclePosition was reported. However on January 3, both TripUpdates and VehiclePosition reported over 70% of trip data.",Yes,both
75,Montebello Schedule,84.68,76.40,NA,NA,NA,NA,NA,found_in_tu_only
98,Santa Ynez Mecatran Schedule,OK,44.57,NA,NA,NA,NA,NA,found_in_tu_only
102,SunLine Avail Schedule,50.84,45.78,NA,NA,NA,NA,NA,found_in_tu_only
